## <span style="color:#ff5f27">👨🏻‍🏫 Build Index </span>

In this notebook you will create a feature group for your candidate embeddings.

## <span style="color:#ff5f27">📝 Imports </span>

In [1]:
import tensorflow as tf
import pprint
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

## <span style="color:#ff5f27">🔮 Connect to Hopsworks Feature Store </span>

In [2]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()
mr = project.get_model_registry()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://snurran.hops.works/p/17527
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


## <span style="color:#ff5f27">🎯 Compute Candidate Embeddings </span>

You start by computing candidate embeddings for all items in the training data.

First, you load your candidate model. Recall that you uploaded it to the Hopsworks Model Registry in the previous notebook. If you don't have the model locally you can download it from the Model Registry using the following code:

In [3]:
model = mr.get_model(
    name="candidate_model",
    version=1,
)
model_path = model.download()

If you already have the model saved locally you can simply replace `model_path` with the path to your model.

In [4]:
candidate_model = tf.saved_model.load(model_path)

Next you compute the embeddings of all candidate items that were used to train the retrieval model.

In [5]:
feature_view = fs.get_feature_view(
    name="retrieval", 
    version=1,
)

In [6]:
train_df, val_df, test_df, _, _, _ = feature_view.train_validation_test_split(
    validation_size=0.1, 
    test_size=0.1,
    description='Retrieval dataset splits',
)

Finished: Reading data from Hopsworks, using ArrowFlight (221.05s) 


In [7]:
train_df.head(3)

,customer_id,article_id,t_dat,price,month_sin,month_cos,age,club_member_status,age_group,garment_group_name,index_group_name
0,d32f22fb2dd1db40ee1bd9e1c214417be617244e8d169e...,663383001,1543190400000,0.030492,-0.500000,0.866025,32.0,ACTIVE,26-35,Jersey Fancy,Baby/Children
1,a111f6bbda7f5e34aa55cc01a94330618e0b8e623ed5b3...,636587016,1543017600000,0.006763,-0.500000,0.866025,66.0,ACTIVE,66+,Jersey Basic,Menswear
2,976b4e7bf5822c869e6638b81bc630350d7c891165b943...,556560005,1540684800000,0.015237,-0.866025,0.500000,44.0,ACTIVE,36-45,Trousers Denim,Baby/Children


In [8]:
# Get the list of input features for the candidate model from the model schema
model_schema = model.model_schema['input_schema']['columnar_schema']
candidate_features = [feat['name'] for feat in model_schema]

# Select the candidate features from the training DataFrame
item_df = train_df[candidate_features]

# Drop duplicate rows based on the 'article_id' column to get unique candidate items
item_df.drop_duplicates(subset="article_id", inplace=True)

item_df.head(3)

,article_id,garment_group_name,index_group_name
0,663383001,Jersey Fancy,Baby/Children
1,636587016,Jersey Basic,Menswear
2,556560005,Trousers Denim,Baby/Children


In [9]:
# Create a TensorFlow dataset from the item DataFrame
item_ds = tf.data.Dataset.from_tensor_slices(
    {col: item_df[col] for col in item_df})

# Compute embeddings for all candidate items using the candidate_model
candidate_embeddings = item_ds.batch(2048).map(
    lambda x: (x["article_id"], candidate_model(x))
)

> Strictly speaking, you haven't actually computed the candidate embeddings yet, as the dataset functions are lazily evaluated.

## <span style="color:#ff5f27">⚙️ Data Preparation </span>


In [10]:
# Concatenate all article IDs and embeddings from the candidate_embeddings dataset
all_article_ids = tf.concat([batch[0] for batch in candidate_embeddings], axis=0)
all_embeddings = tf.concat([batch[1] for batch in candidate_embeddings], axis=0)

# Convert tensors to numpy arrays
all_article_ids_np = all_article_ids.numpy().astype(int)
all_embeddings_np = all_embeddings.numpy()

# Convert numpy arrays to lists
items_ids_list = all_article_ids_np.tolist()
embeddings_list = all_embeddings_np.tolist()

In [11]:
# Create a DataFrame
data_emb = pd.DataFrame({
    'article_id': items_ids_list, 
    'embeddings': embeddings_list,
})

data_emb.head()

,article_id,embeddings
0,663383001,"[101.51985931396484, 64.09200286865234, 88.867..."
1,636587016,"[102.14459228515625, 64.44893646240234, 88.727..."
2,556560005,"[102.36914825439453, 64.44371032714844, 88.646..."
3,665851003,"[102.3536605834961, 63.86991500854492, 88.8890..."
4,636938001,"[102.42069244384766, 63.75370788574219, 88.964..."


## <span style="color:#ff5f27">🪄 Feature Group Creation </span>

Now you are ready to create a feature group for your candidate embeddings.

To begin with, you need to create your Embedding Index where you will specify the name of the embeddings feature and the embeddings length.
Then you attach this index to the FG.

In [12]:
from hsfs import embedding

# Create the Embedding Index
emb = embedding.EmbeddingIndex()

emb.add_embedding(
    "embeddings",                           # Embeddings feature name
    len(data_emb["embeddings"].iloc[0]),    # Embeddings length
)

In [13]:
# Get or create the 'candidate_embeddings_fg' feature group
candidate_embeddings_fg = fs.get_or_create_feature_group(
    name="candidate_embeddings_fg",
    embedding_index=emb,                    # Specify the Embedding Index
    primary_key=['article_id'],
    version=1,
    description='Embeddings for each article',
    online_enabled=True,
)

candidate_embeddings_fg.insert(data_emb)

Feature Group created successfully, explore it at 
https://snurran.hops.works/p/17527/fs/17475/fg/20556


Uploading Dataframe: 0.00% |          | Rows 0/42854 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: candidate_embeddings_fg_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://snurran.hops.works/p/17527/jobs/named/candidate_embeddings_fg_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7f1b0f57f6d0>, None)

## <span style="color:#ff5f27">🪄 Feature View Creation </span>


In [14]:
# Get or create the 'candidate_embeddings' feature view
feature_view = fs.get_or_create_feature_view(
    name="candidate_embeddings",
    version=1,
    description='Embeddings of each article',
    query=candidate_embeddings_fg.select(["article_id"]),
)

Feature view created successfully, explore it at 
https://snurran.hops.works/p/17527/fs/17475/fv/candidate_embeddings/version/1


---
## <span style="color:#ff5f27">⏩️ Next Steps </span>

At this point you have a recommender system that is able to generate a set of candidate items for a customer. However, many of these could be poor, as the candidate model was trained with only a few subset of the features. In the next notebook, you'll create a ranking dataset to train a *ranking model* to do more fine-grained predictions.